In [1]:
from numpy import array
from numpy.linalg import inv

In [ ]:
# Set extrinsic translation matrices based on physical measurements, no z translation assumed
T_l2t = array([[1, 0, 0, 0.028],
                [0, 1, 0, -0.038],
                [0, 0, 1, 0],
                [0, 0, 0, 1]])
T_l2w = array([[1, 0, 0, 0.083],
                [0, 1, 0, -0.035],
                [0, 0, 1, 0],
                [0, 0, 0, 1]])

# Set extrinsic rotation matrices from stereo calibration
R_t2cₜ = array([[0.804905, 0.593319, 0.010014],
                [-0.588094, 0.795337, 0.146920],
                [0.079206, -0.124146, 0.989098]])
R_l2cₜ = array([[0.813639, 0.571181, 0.108367],
                [-0.580035, 0.784919, 0.217856],
                [0.039376, -0.240112, 0.969946]])
R_w2cᵣ = array([[0.903012, -0.397065, -0.164039],
                [0.397183, 0.917127, -0.033513],
                [0.163751, -0.034891, 0.985884]])
R_l2cᵣ = array([[0.909488, -0.399788, -0.114025],
                [0.399705, 0.916314, -0.024592],
                [0.114314, -0.023211, 0.993173]])

# Set intrinsic matrices for the three sensors
Kₗ = array([[205.046875, 0.0, 107.55435943603516],
            [0.0, 205.046875, 82.43924713134766],
            [0.0, 0.0, 1.0]])
Kₜ = array([[161.393925, 0.000000, 78.062273],
            [0.000000, 161.761028, 59.925115], 
            [0.000000, 0.000000, 1.000000]])
Kᵣ = array([[446.423112, 0.000000, 163.485603], 
            [0.000000, 446.765896, 131.217485],
            [0.000000, 0.000000, 1.000000]])


In [ ]:
# Get depth at center of LiDAR image
original_lidar_image = 'placeholder - CV image here'
uₗ = 1 #NOTE: put actual coordinate here
vₗ = 1 #NOTE: put actual coordinate here
zₗ = original_lidar_image[round(vₗ),round(uₗ)]

# If depth is not registering (zero), then skip
if zₗ > 1E-8:
    # Calculate the 3D physical coordinate of the center of the LiDAR image
    pₗ = array([uₗ, vₗ, 1])
    l̂ₗ = inv(Kₗ) @ pₗ
    r̄ₗ = zₗ * l̂ₗ
    
    # Perform extrinsic translations to the thermal sensor and webcam
    r̄ₜ = (inv(R_t2cₜ) @ (R_l2cₜ @ r̄ₗ)) + array([T_l2t[0, 3], T_l2t[1, 3], 0]).T
    r̄ᵣ = (inv(R_w2cᵣ) @ (R_l2cᵣ @ r̄ₗ)) + array([T_l2w[0, 3], T_l2w[1, 3], 0]).T
    
    # Transform 3D coordinate to thermal and webcam pixel coordinates
    r̃ₜ = array([r̄ₜ[0]/r̄ₜ[2], r̄ₜ[1]/r̄ₜ[2], r̄ₜ[2]/r̄ₜ[2]])
    r̃ᵣ = array([r̄ᵣ[0]/r̄ᵣ[2], r̄ᵣ[1]/r̄ᵣ[2], r̄ᵣ[2]/r̄ᵣ[2]])
    pₜ = Kₜ @ r̃ₜ
    pᵣ = Kᵣ @ r̃ᵣ
    uₜ, vₜ = pₜ[0], pₜ[1]
    uᵣ, vᵣ = pᵣ[0], pᵣ[1]